In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

## ReadDataset

In [2]:
arr = np.genfromtxt('tic-tac-toe.data', delimiter=',', dtype='<U1')

## Shuffle

In [3]:
random_state = 0

np.random.seed(random_state)
np.random.shuffle(arr)

X = arr[:, :9]
y = arr[:, 9]

X.shape, y.shape

((958, 9), (958,))

## LabelEncoder

In [4]:
feature = np.unique(X)
inx = np.arange(feature.shape[0])
search = np.searchsorted(feature, X)
X = inx[search]

uni = np.unique(y)
inx = np.arange(uni.shape[0])
search = np.searchsorted(uni, y)
y = inx[search].reshape(-1, 1)

X.shape, y.shape

((958, 9), (958, 1))

## OneHotEncoder

In [5]:
n_feature = feature.shape[0]
one_hot = np.zeros((X.shape[0], X.shape[1], n_feature))
for i, unique_value in enumerate(np.unique(X)):
    one_hot[:, :, i][X == unique_value] = 1

X = one_hot.reshape(-1, n_feature*X.shape[1])

X.shape

(958, 27)

## Split

In [6]:
train_split = 0.7
sample = int(train_split*arr.shape[0])

X_train, X_test = X[:sample], X[sample:]
y_train, y_test = y[:sample], y[sample:]

X_train.shape, X_test.shape

((670, 27), (288, 27))

## Model

In [7]:
model = Sequential()

model.add(Dense(8, activation='relu', input_shape=(X_train.shape[1:])))
model.add(Dense(16, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 224       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________


## Train

In [8]:
batch_size = 16
epochs = 50

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Train on 670 samples, validate on 288 samples
Epoch 1/50
670/670 [==============================] - 0s 201us/sample - loss: 0.6549 - acc: 0.6493 - val_loss: 0.6416 - val_acc: 0.6632
Epoch 2/50
670/670 [==============================] - 0s 73us/sample - loss: 0.6416 - acc: 0.6493 - val_loss: 0.6276 - val_acc: 0.6632
Epoch 3/50
670/670 [==============================] - 0s 68us/sample - loss: 0.6309 - acc: 0.6493 - val_loss: 0.6161 - val_acc: 0.6701
Epoch 4/50
670/670 [==============================] - 0s 71us/sample - loss: 0.6199 - acc: 0.6507 - val_loss: 0.6012 - val_acc: 0.6771
Epoch 5/50
670/670 [==============================] - 0s 65us/sample - loss: 0.6049 - acc: 0.6731 - val_loss: 0.5890 - val_acc: 0.6875
Epoch 6/50
670/670 [==============================] - 0s 70us/sample - loss: 0.5905 - acc: 0.6970 - val_loss: 0.5751 - val_acc: 0.7049
Epoch 7/50
670/670 [==============================] - 0s 63us/sample - loss: 0.5725 - acc: 0.7284 - val_loss: 0.5625 - val_acc: 0.7292
Epoch 8/